In [1]:
#importing the libraries

import pandas as pd
import numpy as np
import datetime
import os

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import mlflow
import boto3
import dagshub



In [2]:
def read_data():
    dfs = []
    for i in range(1, 13):  # Range updated for all 12 months of the year
        data = pd.read_parquet(f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-{str(i).zfill(2)}.parquet")
        dfs.append(data)

    data = pd.DataFrame()

    # Iterate over the list of DataFrames
    for df in dfs:
        data = pd.concat([data, df], ignore_index=True)

    return df


In [3]:
def preprocessing(data):
    #lets use the 237 locationID since its the heavy volume place 
    data = data[data['PULocationID']==237]

    #taking the date and the number of trips per day
    data['date'] = data['tpep_pickup_datetime'].dt.date

    Final_data = pd.DataFrame(data.groupby('date')['VendorID'].count())

    Final_data.reset_index(inplace=True)

    Final_data.columns = ['Date', 'No of trips']

    return Final_data

In [4]:
def window_size(Final_data):
    mlflow.set_experiment("best_window_size_237")
    for window_size in range(1,20):
        with mlflow.start_run():
            # Create empty lists to store the input and output data
            X = []
            y = []
            for i in range(window_size, len(Final_data)):
                # Extract the 12-day sequence as input
                input_sequence = Final_data['No of trips'].iloc[i-window_size:i].tolist()
                
                # Extract the 13th day count as output
                output_value = Final_data['No of trips'].iloc[i]
                
                # Append the input and output to the respective lists
                X.append(input_sequence)
                y.append(output_value)

            # Create a new DataFrame from the input and output lists
            X_df = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(window_size)])
            y_df = pd.DataFrame({'y': y})
            new_df = pd.concat([X_df, y_df], axis=1)


            X = new_df.iloc[:, :-1]
            y = new_df['y']

            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Convert the data into DMatrix format for XGBoost
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dtest = xgb.DMatrix(X_test, label=y_test)

            # Set the parameters for the XGBoost model
            params = {
                'objective': 'reg:squarederror',  # Use squared error as the objective for regression
                'eval_metric': 'rmse'  # Root Mean Squared Error (RMSE) as the evaluation metric
            }

            # Train the XGBoost model
            num_rounds = 100  # Number of boosting rounds
            model = xgb.train(params, dtrain, num_rounds)

            # Make predictions on the test set
            y_pred = model.predict(dtest)

            # Calculate RMSE on the test set
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            mlflow.log_metric('Window_size', f"{window_size}")
            mlflow.log_metric("rmse", rmse)
    mlflow.end_run()

In [5]:
def best_window_size(Final_data):
    # Create empty lists to store the input and output data
    X = []
    y = []
    for i in range(7, len(Final_data)):
        # Extract the 7-day sequence as input
        input_sequence = Final_data['No of trips'].iloc[i-7:i].tolist()
        
        # Extract the 8th day count as output
        output_value = Final_data['No of trips'].iloc[i]
        
        # Append the input and output to the respective lists
        X.append(input_sequence)
        y.append(output_value)

    # Create a new DataFrame from the input and output lists
    X_df = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(7)])
    y_df = pd.DataFrame({'y': y})
    new_df = pd.concat([X_df, y_df], axis=1)


    X = new_df.iloc[:, :-1]
    y = new_df['y']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert the data into DMatrix format for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    return X_train, X_test, y_train, y_test

In [6]:
def training_model(X_train, X_test, y_train, y_test):
    mlflow.set_experiment("best_model_237")
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_test, label=y_test)
    def objective(params):
        with mlflow.start_run():
            mlflow.set_tag("model", "xgboost")
            mlflow.log_params(params)
            booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=10,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
            y_pred = booster.predict(valid)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}
    
    search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
    }

    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=Trials()
    )
    mlflow.end_run()

In [7]:
def best_model(X_train, X_test, y_train, y_test):
    mlflow.set_experiment("Final_Model_Total_trips_prediction_237")
    with mlflow.start_run():
        train = xgb.DMatrix(X_train, label=y_train)
        valid = xgb.DMatrix(X_test, label=y_test)
        best_params = {
            'learning_rate': 0.3186829398830741,
            'max_depth': 29,
            'min_child_weight': 6.747015358253847,
            'objective': 'reg:linear',
            'reg_alpha': 0.03259110542981609,
            'reg_lambda': 0.04013136677983584,
            'seed': 42
        }

        mlflow.log_params(best_params)

        booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [8]:
def main_flow():
    # MLflow settings
    dagshub.init("MLOPS_Project_2023", "manojkotary", mlflow=True)
    MLFLOW_TRACKING_URI = "https://dagshub.com/manojkotary/MLOPS_Project_2023.mlflow"

    # Load
    data = read_data()

    # Transform
    Final_data = preprocessing(data)
    print("Final data extracted")
    
    #logging the best window size
    window_size(Final_data)
    print("best window logged")

    # Train and test data post best window size
    X_train, X_test, y_train, y_test = best_window_size(Final_data)
    print("splitted the data based on best window")

    #model selection 
    training_model(X_train, X_test, y_train, y_test)
    print("best hyper parameter logged")

    #building the best model
    best_model(X_train, X_test, y_train, y_test)
    print("trained the best model")

if __name__ == "__main__":
    main_flow()

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=c979f1e4-7dcb-4460-85a1-93d683377270&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=14ae7e04a8afa87ea3f97f3bf1862721538a40f9d43bc4e517f07a1c58e6d95e




Repository initialized!

C:\Users\ManojKumar\AppData\Local\Temp\ipykernel_19044\1949026399.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['tpep_pickup_datetime'].dt.date


...

[10:42:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:3708.49107                        
[1]	validation-rmse:2770.43103                        
[2]	validation-rmse:2182.63741                        
[3]	validation-rmse:1800.85131                        
[4]	validation-rmse:1606.55118                        
[5]	validation-rmse:1510.42259                        
[6]	validation-rmse:1439.29709                        
[7]	validation-rmse:1411.21362                        
[8]	validation-rmse:1387.44257                        
[9]	validation-rmse:1371.10844                        
[10:42:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:sq

2023/07/12 10:44:12 INFO mlflow.tracking.fluent: Experiment with name 'Final_Model_Total_trips_prediction_237' does not exist. Creating a new experiment.


[10:44:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:3977.41302
[1]	validation-rmse:2885.83188
[2]	validation-rmse:2332.32385
[3]	validation-rmse:1834.55608
[4]	validation-rmse:1508.49024
[5]	validation-rmse:1241.92757
[6]	validation-rmse:1054.90712
[7]	validation-rmse:1010.84581
[8]	validation-rmse:1007.84779
[9]	validation-rmse:940.65195
[10]	validation-rmse:889.25075
[11]	validation-rmse:907.39560
[12]	validation-rmse:898.36423
[13]	validation-rmse:880.88666
[14]	validation-rmse:877.99976
[15]	validation-rmse:854.29034
[16]	validation-rmse:836.59985
[17]	validation-rmse:838.18769
[18]	validation-rmse:825.95102
[19]	validation-rmse:826.57171
[20]	validation-rmse:810.82485
[21]	validation-rmse:824.30694
[22]	validation-rmse:842.22486
[23]	validation-rmse:845.14914
[24]	validation-rmse:836.1

c:\Users\ManojKumar\miniconda3\envs\exp-tracking-env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


trained the best model
